In [1]:
import mlflow
from mlflow.tracking import MlflowClient
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
import datetime as dt
from pyspark.ml.feature import OneHotEncoder, VectorAssembler

In [2]:
%scala
val tags = com.databricks.logging.AttributionContext.current.tags
val username = tags.getOrElse(com.databricks.logging.BaseTagDefinitions.TAG_USER, java.util.UUID.randomUUID.toString.replace("-", ""))
spark.conf.set("com.databricks.demo.username", username)

tags: Map[com.databricks.logging.TagDefinition,String] = Map(TagDefinition(opId,) -> HttpServer-6fa0b428ebacab82, TagDefinition(opTarget,) -> /notebook/1500231256277984/command/1500231256277997, TagDefinition(clusterMemory,) -> 18432, TagDefinition(notebookId,) -> 1500231256277984, TagDefinition(projectName,) -> webapp, TagDefinition(eventWindowTime,) -> 6016597.85000002, TagDefinition(httpTarget,) -> /notebook/1500231256277984/command/1500231256277997, TagDefinition(buildHash,) -> "", TagDefinition(browserHash,) -> #notebook/1500231256277984/command/1500231256277999, TagDefinition(host,Host where the request is coming from.) -> 10.139.64.5, TagDefinition(notebookLanguage,) -> python, TagDefinition(sparkVersion,) -> 5.3.x-scala2.11, TagDefinition(hostName,) -> cons-webapp-0, TagDefinition(httpMethod,) -> POST, TagDefinition(browserIdleTime,) -> 2064, TagDefinition(browserTabId,) -> afdb42f8-a5bb-4827-bda7-41e026cb4520, TagDefinition(sourceIpAddress,) -> 73.193.17.210, TagDefinition(browserUserAgent,) -> Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36, TagDefinition(orgId,) -> 1559977908724754, TagDefinition(userAgent,) -> Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36, TagDefinition(clusterId,) -> 0703-220934-empty762, TagDefinition(rootOpId,Unique identifier for a root of a trace (tree of ops).) -> 7776927338635621033_6765987163455679120_ac80bf1b6714425cbf23248de5803c9c, TagDefinition(sessionId,) -> cb5ebdf37f0ec6186cfcff06aeb65e75c192aad6f8a3fb73555a6bd649d95ccc, TagDefinition(clusterCreator,) -> Webapp, TagDefinition(clientBranchName,) -> 2.102.1091, TagDefinition(clusterType,) -> spot, TagDefinition(browserHasFocus,) -> true, TagDefinition(userId,) -> 8499602646880646, TagDefinition(browserIsHidden,) -> false, TagDefinition(opType,) -> HttpServer, TagDefinition(user,) -> ryan.chynoweth@insight.com, TagDefinition(browserHostName,) -> westus.azuredatabricks.net, TagDefinition(parentOpId,) -> ServiceMain-6fa0b428e2bf0002, TagDefinition(jettyRpcType,) -> "com.databricks.backend.common.rpc.InternalDriverBackendMessages$DriverBackendRequest")
username: String = ryan.chynoweth@insight.com

In [3]:
client = MlflowClient() # client
exps = client.list_experiments() # get all experiments

In [4]:
exp = [s for s in exps if "/Users/{}/AzureML_MLFlow/MLFlowExp".format(spark.conf.get("com.databricks.demo.username")) in s.name][0] # get only the exp we want
exp_id = exp.experiment_id # save exp id to variable
artifact_location = exp.artifact_location # artifact location for storing
run = client.create_run(exp_id) # create the run
run_id = run.info.run_id # get the run id

In [5]:
# start and mlflow run
mlflow.start_run(run_id)

In [6]:
spark.read.format("csv").option("inferSchema", "True").option("header", "True").load("/databricks-datasets/bikeSharing/data-001/day.csv")

Out[ 45 ]: DataFrame[instant: int, dteday: timestamp, season: int, yr: int, mnth: int, holiday: int, weekday: int, workingday: int, weathersit: int, temp: double, atemp: double, hum: double, windspeed: double, casual: int, registered: int, cnt: int]

In [7]:
try: 
  df = (spark
        .read
        .format("csv")
        .option("inferSchema", "True")
        .option("header", "True")
        .load("/databricks-datasets/bikeSharing/data-001/day.csv")
       )
  # split data
  train_df, test_df = df.randomSplit([0.7, 0.3])

  # One Hot Encoding
  mnth_encoder = OneHotEncoder(inputCol="mnth", outputCol="encoded_mnth")
  weekday_encoder = OneHotEncoder(inputCol="weekday", outputCol="encoded_weekday")

  # set the training variables we want to use
  train_cols = ['encoded_mnth', 'encoded_weekday', 'temp', 'hum']

  # convert cols to a single features col
  assembler = VectorAssembler(inputCols=train_cols, outputCol="features")

  # Set linear regression model
  lr = LinearRegression(featuresCol="features", labelCol="cnt")

  # Create pipeline
  pipeline = Pipeline(stages=[
      mnth_encoder,
      weekday_encoder,
      assembler,
      lr
  ])

  # fit pipeline
  lrPipelineModel = pipeline.fit(train_df)

  # write model to datetime folder and latest folder
  lrPipelineModel.write().overwrite().save("{}/latest/bike_sharing_model.model".format(artifact_location))
  lrPipelineModel.write().overwrite().save("{}/year={}/month={}/day={}/bike_sharing_model.model".format(artifact_location, dt.datetime.utcnow().year, dt.datetime.utcnow().month, dt.datetime.utcnow().day))

  # write test predictions to datetime and lastest folder
  predictions = lrPipelineModel.transform(test_df)
  predictions.write.format("parquet").mode("overwrite").save("{}/latest/test_predictions.parquet".format(artifact_location))
  predictions.write.format("parquet").mode("overwrite").save("{}/year={}/month={}/day={}/test_predictions.parquet".format(artifact_location, dt.datetime.utcnow().year, dt.datetime.utcnow().month, dt.datetime.utcnow().day))

  # mlflow log evaluations
  evaluator = RegressionEvaluator(labelCol = "cnt", predictionCol = "prediction")

  mlflow.log_metric("mae", evaluator.evaluate(predictions, {evaluator.metricName: "mae"}))
  mlflow.log_metric("rmse", evaluator.evaluate(predictions, {evaluator.metricName: "rmse"}))
  mlflow.log_metric("r2", evaluator.evaluate(predictions, {evaluator.metricName: "r2"}))
  mlflow.set_tag("Model Path", "{}/year={}/month={}/day={}".format(artifact_location, dt.datetime.utcnow().year, dt.datetime.utcnow().month, dt.datetime.utcnow().day))

  mlflow.end_run(status="FINISHED")
  print("Model training finished successfully")
except Exception as e:
  mlflow.log_param("Error", str(e))
  mlflow.end_run(status="FAILED")
  print("Model training failed: {}".format(str(e)))

399193c422da474ab5bdcd48ada5da04
Model training finished successfully